<div style="display: flex; background-color: RGB(255,114,0);" >
<div>
<img src="img/fraudeur_-_BNPP_PF_-_finale.jpg" width="300"/>
</div>

# PROJET - Comment démasquer les fraudeurs ? <mark>EDA toutes les données</mark>
</div>

par BNP Paribas PF

Lien vers le challenge : https://challengedata.ens.fr/participants/challenges/104/

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 1.CHARGEMENT des données
</div>


In [1]:
from os import getcwd
from os.path import join
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import HTML

import warnings
warnings.filterwarnings("ignore")

from pr_auc_score_SB05ixL import *
from bnp_paribas import *

# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = 1

target = "fraud_flag"

# Récupère le répertoire du programme
execution_path = getcwd().split("projet_bnp_paribas_fraudeurs")[0]
execution_path = join(execution_path, 'projet_bnp_paribas_fraudeurs')

data_set_path = join(execution_path, 'dataset')
data_set_train_file_name = join(data_set_path, "train_complete_encoded_hyper_light_2023_01_19.csv")

print(f"Current execution path : {execution_path}")
print(f"Dataset path : {data_set_path}")

Current execution path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_bnp_paribas_fraudeurs
Dataset path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_bnp_paribas_fraudeurs\dataset


> Chargement des données pré-traitées

In [2]:
# [load_splited_data   ]	INFO  Chargement des données train sources...                          
# [load_splited_data   ]	INFO  (92790, 1981) train données chargées                          
# [load_splited_data   ]	INFO  Séparation de X et y...                          
# [load_splited_data   ]	INFO  split du dataset_path...                          
# [load_splited_data   ]	INFO  5 dataset LOAD                          
# [load_splited_data   ]	INFO  (74232, 1980) / (18558, 1980) 
random_state = 42
dataset_dict = load_splited_data(dataset_path=data_set_train_file_name,over_name="BorderlineSMOTE", random_state = random_state, save_it=True, force=False, verbose=verbose)

X_train, X_test, y_train, y_test, train_origin = dataset_dict.get("X_train", None), dataset_dict.get("X_test", None), dataset_dict.get("y_train", None), dataset_dict.get("y_test", None), dataset_dict.get("train_origin", None)

[load_splited_data   ]	INFO  IN PROGRESS...                          
[load_splited_data   ]	INFO  over BorderlineSMOTE Loading X_train...                          


> Chargement du set de test officiel

In [5]:
# file_name= "test_complete_encoded_2023_01_19.csv"
data_set_test_file_name = data_set_train_file_name.replace("train_complete_hyper", "test_complete")
test_origin = load_test_df(file_path=data_set_test_file_name, train_columns=X_train.columns, verbose=verbose)
dataset_dict["test_origin"] = test_origin
test_origin

(92790, 1981) test données chargées
(92790, 1982) test données mises à jour


,index,ID,Nb_of_items,item_BED LINEN_nb,item_BED LINEN_cash,item_BATHROOM_nb,item_BATHROOM_cash,item_DISPOSABLE TABLEWARE CUTLERY_nb,item_DISPOSABLE TABLEWARE CUTLERY_cash,item_TOYS_nb,...,make_BLUEBELLA_cash,make_ROLSER_nb,make_ROLSER_cash,make_DARTINGTON CRYSTAL_nb,make_DARTINGTON CRYSTAL_cash,make_TEMPLE ISLAND_nb,make_TEMPLE ISLAND_cash,make_GHD_nb,make_GHD_cash,amount
index,,,,,,,,,,,,,,,,,,,,,
0,0,85517,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,889.0
1,0,51113,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,409.0
2,0,83008,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1399.0
3,0,78712,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,808.0
4,0,77846,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,0,21243,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0
92786,0,45891,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,898.0
92787,0,42613,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1727.0


> Chargement des scores déjà enregistrés

In [6]:
score_path = r'C:\Users\User\WORK\workspace-ia\PROJETS\projet_bnp_paribas_fraudeurs\dataset\score.csv'
scores = load_scores(score_path=score_path, save_it=False, verbose=verbose)
print(scores.shape)
scores.sort_values(["pr_auc_score TEST officiel","pr_auc_score TEST perso"], ascending=False)

(22, 8)


,Modèle,Features,Add Data,Accuracy Score,pr_auc_score TEST perso,pr_auc_score TEST officiel,Params,Commentaire
date,,,,,,,,
2023-01-20 16:16,LGBMClassifier,Items,NaN,0.986259,0.093033,0.078557,"boosting_type='goss', max_depth=5,learning_rat...",Sans ajout de données
2023-01-21 17:29,LGBMClassifier,Items,RandomOverSampler,0.925989,0.879887,0.068491,"boosting_type='goss', max_depth=5,learning_rat...",AVEC ajout de données sur tout le dataset
2023-01-21 17:31,LogisticRegression,Items,RandomOverSampler,0.733608,0.657288,0.032113,"penalty=""l2"", fit_intercept=True,solver='libli...",AVEC ajout de données sur tout le dataset
2023-01-20 15:29,LogisticRegression,Items,NaN,0.985074,0.014657,0.016812,"penalty=""l2"", fit_intercept=True,solver='libli...",Sans ajout de données
2023-01-21 17:45,LGBMClassifier,ALL,RandomOverSampler,0.932220,0.888196,0.000000,"boosting_type='goss', max_depth=5,learning_rat...",AVEC ajout de données sur tout le dataset
2023-01-21 17:40,LogisticRegression,ALL,RandomOverSampler,0.726748,0.650713,0.000000,"penalty=""l2"", fit_intercept=True,solver='libli...",AVEC ajout de données sur tout le dataset
2023-01-21 17:35,LGBMClassifier,ALL,NaN,0.986152,0.093013,0.000000,"boosting_type='goss', max_depth=5,learning_rat...",SANS ajout de données
2023-02-14 14:59,LGBMClassifier,ALL,NaN,0.985936,0.078879,0.000000,"boosting_type='goss', max_depth=5,learning_rat...",NaN
2023-02-14 16:56,LGBMClassifier,ALL,NaN,0.985667,0.070795,0.000000,"boosting_type='goss', max_depth=5,learning_rat...",add amount column


<div style="display: flex; background-color: Blue; padding: 15px;" >

## 3.Modèles
</div>

In [6]:
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

<div style="display: flex; background-color: Green; padding: 7px;" >

### 3.1. Essai premier modèle sur item uniquement sans traitement
</div>

<div style="display: flex; background-color: indigo;" >

#### 3.1.1. LogisticRegression
</div>

In [57]:
# accuracy score : 0.9848043970255416, pr_auc score : 0.014656751805151417
# accuracy score : 0.9848582821424723, pr_auc score : 0.014656751805151417
my_fist_model, n_scores = train_LogisticRegression(dataset_dict=dataset_dict, data_set_path=data_set_path, 
                            scores=scores, score_path=score_path, 
                            features="ALL", add_data=np.nan,commentaire="add amount column", 
                            verbose=verbose)

[train_model         ]	INFO  accuracy score : 0.9848582821424723, pr_auc score : 0.014656751805151417                          
[train_model         ]	INFO  Prediction for test orgine...                          


In [58]:
scores = n_scores
save_score(n_scores, score_path=score_path)

<div style="display: flex; background-color: indigo;" >

#### 3.1.2. LGBMClassifier
</div>

In [60]:
# previously       0.9861515249488091 and 0.09301333707479051
# accuracy score : 0.9859359844810863, pr_auc score : 0.07887944011003084
# accuracy score : 0.9856665588964328, pr_auc score : 0.07079502735462112
lgb_classifier, n_scores = train_LGBMClassifier(dataset_dict=dataset_dict, data_set_path=data_set_path, 
                            scores=scores, score_path=score_path, 
                            features="ALL", add_data=np.nan,commentaire="add amount column", 
                            verbose=verbose)

[train_model         ]	INFO  accuracy score : 0.9856665588964328, pr_auc score : 0.07079502735462112                          
[train_model         ]	INFO  Prediction for test orgine...                          


In [61]:
scores = n_scores
save_score(n_scores, score_path=score_path)

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 4.Test des models avec les données ajoutées
</div>

In [8]:
dataset_dict_over = deepcopy(dataset_dict)

In [ ]:
for over_name in OVERED_DATASET_FILE_NAME.keys():
    # Modèle	Features	Add Data
    run_logistic = scores[(scores["Add Data"] == over_name)&(scores["Modèle"] == "LogisticRegression")].shape[0] == 0
    run_sqbmc    = scores[(scores["Add Data"] == over_name)&(scores["Modèle"] == "LGBMClassifier")].shape[0] == 0

    if run_logistic or run_sqbmc:
        print(f"{over_name} logistic = {run_logistic} and sqbmc = {run_sqbmc} \t{surligne_text('processing...', 'blue')}")
        # Chargement des données
        file_name = OVERED_DATASET_FILE_NAME.get(over_name)
        
        path = join(data_set_path,file_name)
        if exists(path):
            dataset_over = load_over_dataset(path, verbose=verbose)
                        
            if dataset_over is not None:
                print(f"{over_name} \t ---> {surligne_text('pre-processing...', 'blue')}")
                
                dataset_over = add_amounts(x_df_input=dataset_over, verbose=verbose)
                y_train_over = complete_y_cols(dataset_over, y_param=dataset_over[target])
                X_train_over = dataset_over.drop(target, axis=1)

                dataset_dict_over['X_train'] = X_train_over
                dataset_dict_over['y_train'] = y_train_over

                if run_logistic:
                    try:
                        model, n_scores = train_LogisticRegression(dataset_dict=dataset_dict_over, data_set_path=data_set_path, 
                                        scores=scores, score_path=score_path, 
                                        features="ALL", add_data=over_name,
                                        verbose=verbose)
                        save_score(n_scores, score_path=score_path)
                        scores = n_scores
                        print(f"{over_name} processing logistic = \t{surligne_text('DONE')}")
                    except Exception as error:
                        print(f"{over_name} processing logistic = \t{surligne_text('FAIL', 'red')} : {error}")
                
                if run_sqbmc:
                    try:
                        lgb_classifier, n_scores = train_LGBMClassifier(dataset_dict=dataset_dict_over, data_set_path=data_set_path, 
                                        scores=scores, score_path=score_path, 
                                        features="ALL",  add_data=over_name,
                                        verbose=verbose)
                        save_score(n_scores, score_path=score_path)
                        scores = n_scores
                        print(f"{over_name} processing sqbmc = \t{surligne_text('DONE')}")
                    except Exception as error:
                        print(f"{over_name} processing sqbmc = \t{surligne_text('FAIL', 'red')} : {error}")
    else:
        print(f"{over_name} \t\t {surligne_text('DONE')}")

<div style="display: flex; background-color: Green; padding: 7px;" >

### 4.2. Ajout de données avec SMOTE :  Synthetic Minority Oversampling Technique
</div>

https://kobia.fr/imbalanced-data-et-machine-learning/

> Le sur-échantillonnage synthétique (SMOTE pour Synthetic Minority Oversampling Technique) est une méthode plus avancée, qui produit des observations minoritaires ressemblantes mais distinctes de celles déjà existantes. Pour plus de précisions, vous pouvez consulter notre article dédié au SMOTE.

https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [69]:
dataset_over_SMOTE = over_dataset_with_SMOTE(X_train,y_train, data_set_path=data_set_train_file_name, 
                     sampling_strategy='minority', random_state=random_state, target = target, verbose=verbose)

X_train_over = dataset_over_SMOTE.drop(target, axis=1)
y_train_over = complete_y_cols(X_train_over, y_param=dataset_over_SMOTE[target])

dataset_dict_over_SMOTE = deepcopy(dataset_dict)
dataset_dict_over_SMOTE['X_train'] = X_train_over
dataset_dict_over_SMOTE['y_train'] = y_train_over

[over_SMOTE          ]	INFO  (146370, 1982), (146370,)                          
[over_SMOTE          ]	INFO  c:\Users\User\WORK\workspace-ia\PROJETS\projet_bnp_paribas_fraudeurs\dataset\train_complete_OVER_SMOTE_minority_encoded_hyper_light_2023_01_19.csv SAVE                          


<div style="display: flex; background-color: indigo;" >

#### M.4.2. LogisticRegression
</div>

In [ ]:
# 0.5454251535725833, 0.025792401467034952
# Fail not enought memory (2.16 GiB)
model, n_scores = train_LogisticRegression(dataset_dict=dataset_dict_over_SMOTE, data_set_path=data_set_path, 
                            scores=scores, score_path=score_path, 
                            features="ALL", add_data="SMOTE",commentaire="Amount and Add data on train", 
                            verbose=verbose)

In [ ]:
scores = n_scores
save_score(n_scores, score_path=score_path)

<div style="display: flex; background-color: indigo;" >

#### M.4.3. LGBMClassifier
</div>

In [91]:
# 0.9706864963896972 and fail
# Fail not enought memory (2.16 GiB)
lgb_classifier, n_scores = train_LGBMClassifier(dataset_dict=dataset_dict_over_SMOTE, data_set_path=data_set_path, 
                            scores=scores, score_path=score_path, 
                            features="ALL",  add_data="SMOTE",commentaire="Amount and Add data on train", 
                            verbose=verbose)

MemoryError: Unable to allocate 2.16 GiB for an array with shape (1982, 146370) and data type float64

In [ ]:
scores = n_scores
save_score(n_scores, score_path=score_path)

<div style="display: flex; background-color: Green; padding: 7px;" >

### 4.2. Ajout de données avec Borderline-SMOTE2
</div>

https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/



In [12]:
dataset_over_BorderlineSMOTE = over_dataset_with_BorderlineSMOTE(X_train,y_train, data_set_path=data_set_train_file_name, 
                    target = target, verbose=verbose)

X_train_over = dataset_over_BorderlineSMOTE.drop(target, axis=1)
y_train_over = complete_y_cols(X_train_over, y_param=dataset_over_BorderlineSMOTE[target])

dataset_dict_over_BorderlineSMOTE = deepcopy(dataset_dict)
dataset_dict_over_BorderlineSMOTE['X_train'] = X_train_over
dataset_dict_over_BorderlineSMOTE['y_train'] = y_train_over

[over_BorderlineSMOTE]	INFO  (146370, 1982), (146370,)                          
[over_BorderlineSMOTE]	INFO  c:\Users\User\WORK\workspace-ia\PROJETS\projet_bnp_paribas_fraudeurs\dataset\train_complete_OVER_BorderlineSMOTE_encoded_hyper_light_2023_01_19.csv SAVE                          


In [ ]:
to_plot_X.append(dataset_dict_over_BorderlineSMOTE["X_train"])
to_plot_y.append(dataset_dict_over_BorderlineSMOTE["y_train"])
titles.append("BorderlineSMOTE")

plot_scatters(Xs=to_plot_X, ys=to_plot_y,titles=titles,target=target, size=(20, 20), verbose=verbose)

<div style="display: flex; background-color: indigo;" >

#### M.4.2. LogisticRegression
</div>

In [ ]:
# 
model, n_scores = train_LogisticRegression(dataset_dict=dataset_dict_over_BorderlineSMOTE, data_set_path=data_set_path, 
                            scores=scores, score_path=score_path, 
                            features="ALL", add_data="BorderlineSMOTE",commentaire="Amount and Add data on train", 
                            verbose=verbose)

In [ ]:
scores = n_scores
save_score(n_scores, score_path=score_path)

<div style="display: flex; background-color: indigo;" >

#### M.4.3. LGBMClassifier
</div>

In [ ]:
# 
lgb_classifier, n_scores = train_LGBMClassifier(dataset_dict=dataset_dict_over_BorderlineSMOTE, data_set_path=data_set_path, 
                            scores=scores, score_path=score_path, 
                            features="ALL",  add_data="BorderlineSMOTE",commentaire="Amount and Add data on train", 
                            verbose=verbose)

In [ ]:
scores = n_scores
save_score(n_scores, score_path=score_path)

<div style="display: flex; background-color: Green; padding: 7px;" >

### 4.2. Ajout de données avec SVMSMOTE
</div>

https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/



In [ ]:
dataset_over_BorderlineSMOTE = over_dataset_with_BorderlineSMOTE(X_train,y_train, data_set_path=data_set_train_file_name, 
                    target = target, verbose=verbose)

X_train_over = dataset_over_BorderlineSMOTE.drop(target, axis=1)
y_train_over = complete_y_cols(X_train_over, y_param=dataset_over_BorderlineSMOTE[target])

dataset_dict_over_BorderlineSMOTE = deepcopy(dataset_dict)
dataset_dict_over_BorderlineSMOTE['X_train'] = X_train_over
dataset_dict_over_BorderlineSMOTE['y_train'] = y_train_over

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 5.Modèles Recherche du modèle le plus adapté
</div>

<div style="display: flex; background-color: Green; padding: 7px;" >

### M.2. Recherche manuelle
</div>

https://towardsdatascience.com/comparing-different-classification-machine-learning-models-for-an-imbalanced-dataset-fdae1af3677f

Logistic Regression, K Nearest Neighbors, Gradient Boosting Classifier, Decision Tree, Random Forest, Neural Net.

<div style="display: flex; background-color: Green; padding: 7px;" >

### M.2. Grid SearCV simple
</div>

In [ ]:
model_dict = {
    
}

<div style="display: flex; background-color: Green; padding: 7px;" >

### M.1. Théories avant réalisation
</div>


Sources : 
- https://blog.octo.com/donnees-desequilibrees-que-faire/

Idées : 
- approche naïve de classification ?  LightGBM et un Stochastic Gradient Descent ?

<div style="display: flex; background-color: indigo;" >

#### M.1.1. Choisir les bonnes métriques
</div>

Métriques : Choisir la métrique la plus pertinente d’un point de vue métier
- <mark>accuracy</mark> non pertinente
- Le <mark>rappel</mark> indique la capacité du modèle à prédire correctement tous les positifs et la précision indique la capacité du modèle à ne prédire que les positifs.
- <mark>score F1</mark> : est très intéressant quand il existe un double enjeu et qu’on cherche à avoir à la fois un bon rappel et une bonne précision.  Il s’agit d’une moyenne des deux métriques
- Le coefficient de corrélation de <mark>Matthews</mark> ou le <mark>MCC</mark> reste la meilleure métrique à utiliser : Concrètement, si on considère que la classe prédite et la classe réelle sont deux variables binaires, le MCC représente la corrélation entre ces deux variables. C’est une métrique qui est symétrique et insensible au déséquilibre de classes.

<div style="display: flex; background-color: indigo;" >

#### M.1.2. Traitement des données
</div>

Techniques de traitement
- Techniques se basant sur l’amélioration du jeu de données
- Techniques se basant sur l’amélioration de la phase de modélisation 


Lorsque le dataset n’est pas assez grand, on peut sur-échantillonner les observations associées à la classe la moins prépondérante. On peut générer de nouveaux échantillons par répétition de façon aléatoire ou en s’appuyant sur des méthodes un peu plus sophistiquées comme SMOTE ou ADASYN

<div style="display: flex; background-color: Blue; padding: 15px;" >

## Z.Annexe
</div>

Idées / Questions :
- Ajouter une colonne montant total
- Ajouter une colonne nombre total d'article ?
- Une catégorie d'article est-elle envisageable ?
- Regroupement par tranche de prix des articles ? des paniers ?
- Est-ce que certains articles sont plus soumis aux fraudes que d'autres ?


|Modèle|Add Data|Features|Params|Accuracy Score|pr_auc_score TEST perso|pr_auc_score TEST officiel|Commentaire|
|------|--------|--------|------|--------------|-----------------------|--------------------------|-----------|
|LogisticRegression||Items|penalty="l2", fit_intercept=True,solver='liblinear'|0.985073822610195|0.014656751805151417|0,0168117941201828|Sans ajout de données|
|LGBMClassifier||Items|boosting_type='goss', max_depth=5,learning_rate=0.1,n_estimators=1000,subsample=0.8,colsample_bytree=0.6|0.9862592951826705|0.09303255321981446|0,07855741945152836|Sans ajout de données|
|LogisticRegression|RandomOverSampler|Items|penalty="l2", fit_intercept=True,solver='liblinear'|0.7336084615594851|0.6572881928321739|0,03211346653991237|AVEC ajout de données sur tout le dataset|
|LGBMClassifier|RandomOverSampler|Items|boosting_type='goss', max_depth=5,learning_rate=0.1,n_estimators=1000,subsample=0.8,colsample_bytree=0.6|0.9259886851239444|0.8798873204426053|0,06849072266219955|AVEC ajout de données sur tout le dataset|
|LogisticRegression||ALL|penalty="l2", fit_intercept=True,solver='liblinear'|0.9849660523763336|0.014656751805151417| |SANS ajout de données|
|LGBMClassifier||ALL|boosting_type='goss', max_depth=5,learning_rate=0.1,n_estimators=1000,subsample=0.8,colsample_bytree=0.6|0.9861515249488091|0.09301333707479051| |SANS ajout de données|
|LogisticRegression|RandomOverSampler|ALL|penalty="l2", fit_intercept=True,solver='liblinear'|0.7267484763180191|0.6507125123342998| |AVEC ajout de données sur tout le dataset|
|LGBMClassifier|RandomOverSampler|ALL|boosting_type='goss', max_depth=5,learning_rate=0.1,n_estimators=1000,subsample=0.8,colsample_bytree=0.6|0.9322200661400968|0.8881964612621873| |AVEC ajout de données sur tout le dataset|
|LogisticRegression|RandomOverSampler|ALL|penalty="l2", fit_intercept=True,solver='liblinear'|0.5651471063692208|0.026699804461301335| |AVEC ajout de données sur Train only|
|LGBMClassifier|RandomOverSampler|ALL|boosting_type='goss', max_depth=5,learning_rate=0.1,n_estimators=1000,subsample=0.8,colsample_bytree=0.6|0.8935230089449294|0.060563220710355574| |AVEC ajout de données sur Train only|

In [26]:
lgb_classifier = lgb.LGBMClassifier(boosting_type='goss',  
                                    max_depth=5, 
                                    learning_rate=0.1,
                                    n_estimators=1000, 
                                    subsample=0.8,  
                                    colsample_bytree=0.6,
                                   )
classifier = lgb_classifier.fit(X_train, y_train, verbose=verbose) 
score_accuracy = classifier.score(X_test, y_test)
print(score_accuracy)
y_pred = classifier.predict(X_test)
pr_auc_score_, y_t_c, y_p_c = evaluate(X_test=X_test, y_test=y_test, y_pred=y_pred, verbose=verbose)
print(pr_auc_score_)

y_pred_test = classifier.predict(test_origin)
y_pred_test_complete = complete_y_cols(X_test=test_origin, y_param=y_pred_test)
res_path = join(data_set_path, "All_LGBMClassifier_"+datetime.now().strftime('%Y-%m-%d-%H_%M')+".csv")
y_pred_test_complete.to_csv(res_path)

n_scores = add_score(scores_param=scores, modele="LGBMClassifier", features="ALL", add_data=np.nan, 
          params="boosting_type='goss', max_depth=5,learning_rate=0.1,n_estimators=1000.subsample=0.8,colsample_bytree=0.6", 
          accuracy_score=score_accuracy, pr_auc_score_TEST_perso=pr_auc_score_,
          commentaire=np.nan,
          score_path=score_path.replace(".csv", f"{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv"),
          verbose=verbose)
n_scores.sort_values(by=['Modèle'])

0.9861515249488091
0.09301333707479051


In [ ]:
X_train = add_amounts(X_train, verbose=verbose)
X_train

,index,ID,Nb_of_items,item_BED LINEN_nb,item_BED LINEN_cash,item_BATHROOM_nb,item_BATHROOM_cash,item_DISPOSABLE TABLEWARE CUTLERY_nb,item_DISPOSABLE TABLEWARE CUTLERY_cash,item_TOYS_nb,...,make_BLUEBELLA_cash,make_ROLSER_nb,make_ROLSER_cash,make_DARTINGTON CRYSTAL_nb,make_DARTINGTON CRYSTAL_cash,make_TEMPLE ISLAND_nb,make_TEMPLE ISLAND_cash,make_GHD_nb,make_GHD_cash,amount
0,12214,79535,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1197.0
1,21619,56408,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1635.0
2,18886,14674,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2639.0
3,85690,33513,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1873.0
4,30417,65918,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74227,6265,5177,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,900.0
74228,54886,77904,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1699.0
74229,76820,83123,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,369.0
74230,860,52208,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1407.0


In [ ]:
X_test = add_amounts(X_test, verbose=verbose)
X_test

,index,ID,Nb_of_items,item_BED LINEN_nb,item_BED LINEN_cash,item_BATHROOM_nb,item_BATHROOM_cash,item_DISPOSABLE TABLEWARE CUTLERY_nb,item_DISPOSABLE TABLEWARE CUTLERY_cash,item_TOYS_nb,...,make_BLUEBELLA_cash,make_ROLSER_nb,make_ROLSER_cash,make_DARTINGTON CRYSTAL_nb,make_DARTINGTON CRYSTAL_cash,make_TEMPLE ISLAND_nb,make_TEMPLE ISLAND_cash,make_GHD_nb,make_GHD_cash,amount
0,46180,107424,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,499.0
1,25031,74645,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100.0
2,32798,102653,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1187.0
3,38656,23831,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,479.0
4,16271,60691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18553,14569,96137,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2610.0
18554,70048,84464,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1419.0
18555,39431,62549,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2849.0
18556,34291,89894,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2499.0


In [ ]:
test_origin = add_amounts(test_origin, verbose=verbose)
test_origin

,index,ID,Nb_of_items,item_BED LINEN_nb,item_BED LINEN_cash,item_BATHROOM_nb,item_BATHROOM_cash,item_DISPOSABLE TABLEWARE CUTLERY_nb,item_DISPOSABLE TABLEWARE CUTLERY_cash,item_TOYS_nb,...,make_BLUEBELLA_cash,make_ROLSER_nb,make_ROLSER_cash,make_DARTINGTON CRYSTAL_nb,make_DARTINGTON CRYSTAL_cash,make_TEMPLE ISLAND_nb,make_TEMPLE ISLAND_cash,make_GHD_nb,make_GHD_cash,amount
index,,,,,,,,,,,,,,,,,,,,,
0,0,85517,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,889.0
1,0,51113,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,409.0
2,0,83008,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1399.0
3,0,78712,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,808.0
4,0,77846,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,0,21243,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306.0
92786,0,45891,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,898.0
92787,0,42613,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1727.0
